In [1]:
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import json
from pathlib import Path
import numpy as np
import os
from os import listdir
from os.path import isfile, join

Lets define some usefull functions we will use later:

In [2]:
def check_or_save_page(filename, url):
    """
        Check if the file exist, if not get the page
        from the url and store in on the disk
        Returns the file content as a soup
    """
    # Check if the page has been stored on disk
    if Path(filename).is_file() is False:
        #print('No page')
        # Get the page
        result = requests.get(url)
        with open(filename,'w') as outfile:
            outfile.write(result.text)
        time.sleep(2)
    #else:
        #print('We got it')
        
    #with open(filename) as my_file:
    with open(filename, encoding='utf-8') as my_file:
        soup = BeautifulSoup(my_file.read(), "html.parser")
        
    return soup

# Data Collection - Web Scraping - Data Parsing 

In [3]:
# Declare global variables
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming', 'District of Columbia']

# National unemployement rate by month from 1948 to 2018
# Source: https://data.bls.gov/pdq/SurveyOutputServlet
national_unemployement_rate = pd.read_csv('data/national_unemployement_1948_2018.csv')

# Get the presidental job approval
# Source: https://www.gallup.com
# https://news.gallup.com/interactives/185273/presidential-job-approval-center.aspx
with open('data/all_presidential_job_approval_gallup.json') as f:
    presidential_approval = json.load(f)
presidential_approval = presidential_approval['AllPresidents']['HistoricalPresident']
presidential_approval_df = pd.DataFrame.from_dict([x['PresidentData'] for x in presidential_approval])
display(presidential_approval_df.head())

,DatesinOffice,DaysInOffice,EndDate,FirstTermAverage,JobApprovalHigh,JobApprovalLow,OverallAverage,Party,PresidentName,SecondTermAverage,StartDate
0,2017-Present,658,,-,45.0,35.0,39.5,Rep.,Donald J. Trump,-,2017-01-20
1,2009-2017,2922,2017-01-20,48,67,40,48,Dem.,Barack Obama,47,2009-01-20
2,2001-2009,2922,2009-01-20,62.2,90,25,49.4,Rep.,George W. Bush,36.5,2001-01-20
3,1993-2001,2922,2001-01-20,49.6,73,37,55.1,Dem.,Bill Clinton,60.6,1993-01-20
4,1989-1993,1461,1993-01-20,60.9,89,29,60.9,Rep.,George H. W. Bush,-,1989-01-20


In [4]:
display(national_unemployement_rate.head())

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1948,3.4,3.8,4.0,3.9,3.5,3.6,3.6,3.9,3.8,3.7,3.8,4.0
1,1949,4.3,4.7,5.0,5.3,6.1,6.2,6.7,6.8,6.6,7.9,6.4,6.6
2,1950,6.5,6.4,6.3,5.8,5.5,5.4,5.0,4.5,4.4,4.2,4.2,4.3
3,1951,3.7,3.4,3.4,3.1,3.0,3.2,3.1,3.1,3.3,3.5,3.5,3.1
4,1952,3.2,3.1,2.9,2.9,3.0,3.0,3.2,3.4,3.1,3.0,2.8,2.7


In [5]:
# Get once the necessary pages
presidential_page = requests.get('https://en.wikipedia.org/wiki/United_States_presidential_election')

In [6]:
# List of the US presidents
president_elected_history = pd.read_csv('data/president_elected_history.csv', sep=';')
display(president_elected_history.head())

,year,president_elected,president_elected_party,can_be_re_elected
0,1824,John Quincy Adams,DR,1
1,1828,Andrew Jackson,D,1
2,1832,Andrew Jackson,D,0
3,1836,Martin Van Buren,D,1
4,1840,William Henry Harrison,W,1


In [7]:
"""
Get the national level factors
Source: https://en.wikipedia.org/wiki/United_States_presidential_election
"""

# From a tag, extract the number of seats
def extract_seats(tag):
    if tag.findAll('b'):
        d_seats = tag.b.extract().string
    elif tag.sup and tag.sup.decompose():
        d_seats = tag.sup.decompose()
    elif tag.string is None:
        d_seats = tag.text
    else:
        d_seats = tag.string
    return int(d_seats)

def extract_seats_change(tag):
    if tag.sup:
        d_seats_change = tag.text.split('[', 1)[0]
    else:
        d_seats_change = tag.text
    return int(d_seats_change.replace('–', '-'))

# Get the house election years
def extract_house_elections_history():
    house_elections_history = []

    # If the file doesn't exist, get the data from the webpage and store the content to a new file
    filename = 'data/list_of_house_elections_page.html'
    if Path(filename).is_file():
        #with open(filename) as my_file:
        with open(filename, encoding='utf-8') as my_file:
            list_of_house_elections_page = my_file.read()
    else:
        print('no file')
        list_of_house_elections_page = requests.get('https://en.wikipedia.org/wiki/List_of_United_States_House_of_Representatives_elections,_1856%E2%80%93present')
        with open(filename,'w') as outfile:
            outfile.write(list_of_house_elections_page.text)

    soup = BeautifulSoup(list_of_house_elections_page, "html.parser")

    # Find the election years
    data = []
    elections_pages = []
    for t in soup.find_all('a', title=lambda x: x and 'United States House of Representatives elections,' in x):
        if len(t.string) == 4:
            elections_pages.append({
                'year': int(t.string),
                'url':'https://en.wikipedia.org'+t.attrs['href']
            })
            year = int(t.string)+2

            cols = t.parent.parent.find_all('td')

            # Get the number of Democrat seats
            d_seats = extract_seats(cols[1])

            # Get the change in the number of Democrat seats
            d_seats_change = extract_seats_change(cols[2])

            # Get the number of Republican seats
            r_seats = extract_seats(cols[3])
            
            # Get the change in the number of Republican seats
            r_seats_change_by_year = extract_seats_change(cols[4])
            
            #print(1 if year in presidential_years else 0)

            idx = (np.abs(president_elected_history['year'].values-year+1)).argmin()
            president_can_be_re_elected = president_elected_history['can_be_re_elected'].loc[[idx]].values[0]
            president_party = president_elected_history['president_elected_party'].loc[[idx]].values[0]

            # Look for president overall job approval average
            president_name = president_elected_history['president_elected'].loc[[idx]].values[0]
            president_overall_avg_job_approval = presidential_approval_df.loc[presidential_approval_df['PresidentName'] == president_name]['OverallAverage']
            president_overall_avg_job_approval = float(president_overall_avg_job_approval.values[0])/100 if president_overall_avg_job_approval.values.size else None
            
            # Get the national unemployement rate for November
            oct_unemployement_rate = national_unemployement_rate.loc[national_unemployement_rate['Year'] == year]['Oct']
            
            oct_unemployement_rate = oct_unemployement_rate.values[0] if oct_unemployement_rate.values.size else None
            
            data.append({
                'year': year,
                'is_presidential_year': 1 if year in president_elected_history['year'].unique() else 0,
                'president_party': president_party,
                'president_can_be_re_elected': president_can_be_re_elected,
                'president_overall_avg_job_approval': president_overall_avg_job_approval,
                'oct_unemployement_rate': oct_unemployement_rate,
                'last_democrat_seats': d_seats,
                'last_republican seats': r_seats,
                'last_house_majority': 'R' if d_seats < r_seats else 'D'
            })

    return data, elections_pages

data, house_elections_pages = extract_house_elections_history()
data_df = pd.DataFrame(data)
national_level_factors = data_df[[
    'year', 
    'is_presidential_year', 
    'president_party', 
    'president_can_be_re_elected', 
    'president_overall_avg_job_approval', 
    'oct_unemployement_rate',
    'last_democrat_seats', 
    'last_republican seats', 
    'last_house_majority']]
display(national_level_factors.sort_values('year', ascending=False).head())

,year,is_presidential_year,president_party,president_can_be_re_elected,president_overall_avg_job_approval,oct_unemployement_rate,last_democrat_seats,last_republican seats,last_house_majority
80,2018,0,R,1,0.395,3.7,194,241,R
79,2016,1,R,1,0.395,4.9,188,247,R
78,2014,0,D,0,0.480,5.7,201,234,R
77,2012,1,D,0,0.480,7.8,193,242,R
76,2010,0,D,1,0.480,9.4,257,178,D


In [8]:
"""
Get the state level factors
"""

# Historical presidential election results by state
# Source: https://en.wikipedia.org/wiki/List_of_United_States_presidential_election_results_by_state
election_results_df = pd.read_csv('data/presidential_election_results_by_state.csv')
election_results_df.head()

,State,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,1789,NaN,NaN,NaN,NaN,NaN,NaN,GW,GW,NaN,...,NaN,NaN,NaN,NaN,NaN,GW,NaN,NaN,NaN,NaN
1,1792,NaN,NaN,NaN,NaN,NaN,NaN,GW,GW,NaN,...,NaN,NaN,NaN,NaN,GW,GW,NaN,NaN,NaN,NaN
2,1796,NaN,NaN,NaN,NaN,NaN,NaN,F,F,NaN,...,NaN,DR,NaN,NaN,F,DR,NaN,NaN,NaN,NaN
3,1800,NaN,NaN,NaN,NaN,NaN,NaN,F,F,NaN,...,NaN,DR,NaN,NaN,F,DR,NaN,NaN,NaN,NaN
4,1804,NaN,NaN,NaN,NaN,NaN,NaN,F,F,NaN,...,NaN,DR,NaN,NaN,DR,DR,NaN,NaN,NaN,NaN


## wikipedia.org
### Get the House and Senate election result pages for all the available years

In [9]:
"""
Get the district level factor
"""
def get_district_list():
    district_list = []
    url = 'https://en.wikipedia.org/wiki/United_States_House_of_Representatives_elections,_2016'
    filename = 'data/wikipedia/all_state_districts_list_page.html'
    
    # Check if the page has been stored on disk
    soup = check_or_save_page(filename, url)
    
    # Find the districts page links
    districts = soup.find_all('a', href=re.compile(r'(.*\/wiki\/.* )|(.*_congressional_district)'))
    
    for district in districts:
        if any(substring in district.string for substring in states) \
        and district.string not in district_list \
        and "'s" not in district.string \
        and "12th" not in district.string \
        and '1st' not in district.string:
            district_state = ''
            # Get the corresponding state
            for state in states:
                if state in district.string:
                    district_state = state
            
            # Format the district name
            if 'at-large' in district.string:
                dist_name = 'At-Large'
            else:
                # Find the district number
                dist_number = [int(s) for s in district.string.split() if s.isdigit()]
                if len(dist_number) > 0:
                    dist_number = dist_number[0]
                    dist_name = 'District {}'.format(dist_number)
                else:
                    continue
                
            #print(district_state, dist_name)
            
            district_list.append({
                'name': dist_name,
                'page_url': 'https://en.wikipedia.org{}'.format(district['href']),
                'state': district_state
            })
            
    # Remove duplicate in the list
    district_list = [dict(t) for t in {tuple(d.items()) for d in district_list}]
    
    return district_list

def get_wiki_district_pages(districts):
    # Get the district pages if they have not been stored on disk yet
    for district in districts:
        filename = 'data/district_pages/{}.html'.format(district['name'])

        # Check if the page has been stored on disk
        check_or_save_page(filename, district['page_url'])
        
def parse_district_house_results(filename, district, state):
    undesirable_chars = ['\*', '%', '\(incumbent\)', '\(inc.\)', '\(write-in\)']
    district_house_results = []
     #with open(filename) as my_file:
    with open(filename, encoding='utf-8') as my_file:
        soup = BeautifulSoup(my_file.read(), "html.parser")
        
        # Find the election results tables
        caption = soup.find_all('caption')
        elems = []
        for capt in caption:
            x = capt.get_text()
            if ('United States House of Representatives elections,' in x or
                'congressional district election' in x or
                'US House election, ' in x or
                'Congressional District House Election'
            ):
                elems.append(capt)

        for capt in elems:
            # Find the date
            match = re.match(r'.*([1-2][0-9]{3})', capt.text)
            if match is None:
                continue

            # Then it found a match!
            year = int(match.group(1))
            #print(year)

            # Get the result table itself
            table = capt.find_parent('table')
            table_body = table.find('tbody')
            rows = table_body.find_all('tr')

            for row in rows:
                cols = row.find_all('td')
                cols = [ele.text.strip() for ele in cols]
                cols = [ele for ele in cols if ele] # Get rid of empty values

                if len(cols) and cols[0] in ['Republican', 'Democratic']:
                    print(cols)
                    
                    percent = np.NaN
                    if len(cols) > 3 and cols[3] != 'N/A':
                        percent = float(re.sub("|".join(undesirable_chars), "", cols[3]))/100
                    
                    votes = np.NaN
                    if len(cols) > 2 and cols[2] == 'N/A':
                        votes = np.NaN
                    elif len(cols) > 2 and '%' not in cols[2] and cols[2] != '100.00':
                        votes = int(cols[2].replace(',', '').replace('.', ''))
                    elif len(cols) > 2 and ('%' in cols[2] or cols[2] == '100.00'):
                        percent = float(re.sub("|".join(undesirable_chars), "", cols[2]))/100
                    
                    district_house_results.append({
                        'year': year,
                        'candidate_party': 'R' if cols[0] == "Republican" else 'D',
                        'candidate_name': re.sub("|".join(undesirable_chars), "", cols[1]),
                        'votes': votes,
                        'percent': percent
                    })
                    
    return pd.DataFrame(district_house_results)
    
def get_district_level_factors(district):
    state = districts_df.loc[districts_df['name'] == district]['state'].values[0]
    dist_level_factors = []
    # Get the page of the district
    html_filename = 'data/district_pages/{}.html'.format(district)
    json_filename = 'data/district_pages/{}.json'.format(district)
    
    # If there is no already formated data, get them from the corresponding Wikipedia page
    if Path(json_filename).is_file() is False:
        district_house_results = parse_district_house_results(html_filename, district, state)
        display(district_house_results)
    else:
        print('get from json')
        district_house_results = pd.read_json(json_filename)
        display(district_house_results)
        
    # Now, for each year
    for year in district_house_results['year'].unique():
        # If there is more than 1 candidate this year
        # Get the current year
        if len(district_house_results.loc[district_house_results['year'] == year]) > 1:
            curr_year_idx = district_house_results.loc[district_house_results['year'] == year]['votes'].idxmax()
            curr_year = district_house_results.loc[[curr_year_idx]]
        else:
            curr_year = district_house_results        
        
        # Get previous year
        prev_year = district_house_results.loc[district_house_results['year'] == year-2]

        if prev_year.empty is False:
            # If there is more than 1 candidate the previous year
            if len(prev_year) > 1:
                prev_year_winner_idx = prev_year['votes'].idxmax()
                prev_year_winner = prev_year.loc[[prev_year_winner_idx]]
            else:
                prev_year_winner = prev_year

            # Get the incumbent name
            incumbent = prev_year_winner['candidate_name'].values[0]

            # Get the elections previously won by the incumbent
            incumbent_history = district_house_results.loc[
                (district_house_results['candidate_name'] == incumbent) &
                (district_house_results['year'] < year)
            ]

            incubent_first_elected_idx = incumbent_history['year'].idxmin()
            incubent_first_elected = incumbent_history.loc[[incubent_first_elected_idx]]['year'].values[0]
            incubent_is_candidate = curr_year.loc[curr_year['candidate_name'] == incumbent].empty

            dist_data = {
                'year': year,
                'state': state,
                'district': district,
                'incumbent': incumbent,
                'incumbent_party': 'R' if prev_year_winner['candidate_party'].values[0] == "Republican" else 'D',
                'incumbent_count_victories': len(incumbent_history),
                'incumbent_first_elected': incubent_first_elected,
                'incumbent_running_re_election': 0 if incubent_is_candidate else 1,
                'candidate_elected_party': curr_year['candidate_party'].values[0]
            }

            dist_level_factors.append(dist_data)
        else:
            print('yop')
            dist_data = {
                'year': year,
                'state': state,
                'district': district,
                'incumbent': np.NaN,
                'incumbent_party': np.NaN,
                'incumbent_count_victories': np.NaN,
                'incumbent_first_elected': np.NaN,
                'incumbent_running_re_election': np.NaN,
                'candidate_elected_party': curr_year['candidate_party'].values[0]
            }
            dist_level_factors.append(dist_data)

    return dist_level_factors

#for district in ['Alabama 1', 'Alabama 2']:
#for district in ['Arkansas 1']:
#    district_level_factors = get_district_level_factors(district)
#    display(pd.DataFrame(district_level_factors).sort_values('year', ascending=True))
#    #display(district_level_factors)

def get_wiki_districts_house_results(districts_list):
    candidate_results = []
    wiki_undesirable_chars = [
        '\*', '%', '\(Incumbent\)', '\(incumbent\)', '\(inc.\)', '\(write-in\)', 
        '\(as a write-in\)'
    ]
    for district in districts_list:
        # To remove
        #if district['state'] != 'Texas' or district['name'] != 'District 17':
        #if district['state'] != 'Wyoming':
        #    continue
        
        print('Will get results for house/{}/{}.html'.format(district['state'], district['name']))
        print('Source: {}'.format(district['page_url']))
        
        # In some cases, the wikipedia page is too messy to crawl
        # So I manually gather the informations into a json file
        # If this file exist, it will be prefered
        json_filename = 'data/wikipedia/house/{}/{}.json'.format(district['state'], district['name'])
        if Path(json_filename).is_file() is True:
            print('Data are store in a formated JSON')
            continue
        
        # Create the directories if necessary
        if not os.path.exists('data/wikipedia/house'):
            os.makedirs('data/wikipedia/house')
        if not os.path.exists('data/wikipedia/house/{}'.format(district['state'])):
            os.makedirs('data/wikipedia/house/{}'.format(district['state']))
            
        filename = 'data/wikipedia/house/{}/{}.html'.format(district['state'], district['name'])
        
        # Check if the page has been stored on disk
        soup = check_or_save_page(filename, district['page_url'])
        
        # Find the results tables
        caption = soup.find_all('caption')
        tables = []
        for capt in caption:
            x = capt.get_text()
            if ('United States House of Representatives elections,' in x or
                'congressional district election' in x or
                'US House election, ' in x or
                'Congressional District House Election'
            ):
                # print(capt)
                table = capt.find_parent('table')
                tables.append(table)
        
        # For each result table, extract the results
        for table in tables:
            # Get the year
            table_title = table.find('caption')
            
            # If this is a table about a special election, skip it
            if 'Special' in table_title.text:
                continue
            
            year_match = re.match(r'.*([1-2][0-9]{3})', table_title.text)
            
            # If there is no year match, then this table isn't of interest
            if year_match is None:
                continue
            
            year = int(year_match.group(1))
            # print(year)
            
            # Get the result table itself
            rows = table.find('tbody').find_all('tr')
            candidate_rows = []
            for row in rows:
                cols = row.find_all('td')
                cols = [ele.text.strip() for ele in cols]
                # If all the values of the cols are empty strings, continue
                if all(v is '' for v in cols):
                    continue
                    
                # print(cols)
                
                # If this row contains a candidate results
                if len(cols) > 2 and cols[1] in ['Republican', 'Democratic']:
                    # print(cols)
                    
                    party = 'R' if cols[1] == 'Republican' else 'D'
                    name = cols[2]
                    votes = int(cols[3].replace(',', '').replace('[8]', '').replace('c', '').replace('.', '').replace(' ', '')) if cols[3] != '' else np.NaN
                    percent = float(cols[4].replace('%', '')) if cols[4] != '' else np.NaN
                    
                    candidate_rows.append({
                        'year': year,
                        'state': district['state'],
                        'district': district['name'],
                        'is_incumbent': np.NaN,
                        'name': name,
                        'party': party,
                        'percent': percent,
                        'votes': votes,
                        'won': 0
                    })
                    
            # If we found no candidate data, continue
            if len(candidate_rows) == 0:
                continue
            
            # Enrich the candidates data
            max_percent = max([x['percent'] for x in candidate_rows])
            for candidate in candidate_rows:
                # Check if the candidate won the elections
                if candidate['percent'] == max_percent:
                    candidate['won'] = 1
                    
                # Check if we can determine if the candidate is an incumbent
                if '(inc.)' in candidate['name'] or '(incumbent)' in candidate['name'] or '(Incumbent)' in candidate['name']:
                    candidate['is_incumbent'] = 1

                # Clean the candidate name
                candidate['name'] = re.sub("|".join(wiki_undesirable_chars), "", candidate['name'])
                    
                candidate_results.append(candidate)
                
            # If we found that one of the candidates is an incumbent, the others are sets to 0
            max_incumbent = max([x['is_incumbent'] for x in candidate_rows])
            #print(type(max_incumbent))
            if max_incumbent == 1:
                for candidate in candidate_rows:
                    candidate['is_incumbent'] = 0 if candidate['is_incumbent'] != 1 else 1

    return candidate_results

districts_list = get_district_list()
# districts_df = pd.DataFrame(districts_list)
# display(districts_df.loc[districts_df['state'] == 'Wyoming'])

wiki_house_history = get_wiki_districts_house_results(districts_list)
# Store in disk
wiki_house_history_df = pd.DataFrame(wiki_house_history)
wiki_house_history_df.to_csv('data/wikipedia/house_results.csv', encoding='utf-8')

Will get results for house/Virginia/District 7.html
Source: https://en.wikipedia.org/wiki/Virginia%27s_7th_congressional_district
Will get results for house/Texas/District 12.html
Source: https://en.wikipedia.org/wiki/Texas%27s_12th_congressional_district
Will get results for house/Washington/District 3.html
Source: https://en.wikipedia.org/wiki/Washington%27s_3rd_congressional_district
Will get results for house/Indiana/District 4.html
Source: https://en.wikipedia.org/wiki/Indiana%27s_4th_congressional_district
Will get results for house/New York/District 7.html
Source: https://en.wikipedia.org/wiki/New_York%27s_7th_congressional_district
Will get results for house/New York/District 18.html
Source: https://en.wikipedia.org/wiki/New_York%27s_18th_congressional_district
Will get results for house/Ohio/District 2.html
Source: https://en.wikipedia.org/wiki/Ohio%27s_2nd_congressional_district
Will get results for house/Kentucky/District 3.html
Source: https://en.wikipedia.org/wiki/Kentucky

Will get results for house/Mississippi/District 3.html
Source: https://en.wikipedia.org/wiki/Mississippi%27s_3rd_congressional_district
Will get results for house/California/District 36.html
Source: https://en.wikipedia.org/wiki/California%27s_36th_congressional_district
Will get results for house/Ohio/District 13.html
Source: https://en.wikipedia.org/wiki/Ohio%27s_13th_congressional_district
Will get results for house/Oklahoma/District 1.html
Source: https://en.wikipedia.org/wiki/Oklahoma%27s_1st_congressional_district
Will get results for house/New Jersey/District 6.html
Source: https://en.wikipedia.org/wiki/New_Jersey%27s_6th_congressional_district
Will get results for house/Indiana/District 1.html
Source: https://en.wikipedia.org/wiki/Indiana%27s_1st_congressional_district
Will get results for house/Massachusetts/District 3.html
Source: https://en.wikipedia.org/wiki/Massachusetts%27s_3rd_congressional_district
Will get results for house/Wisconsin/District 7.html
Source: https://en.

Will get results for house/California/District 31.html
Source: https://en.wikipedia.org/wiki/California%27s_31st_congressional_district
Will get results for house/Florida/District 15.html
Source: https://en.wikipedia.org/wiki/Florida%27s_15th_congressional_district
Will get results for house/Pennsylvania/District 14.html
Source: https://en.wikipedia.org/wiki/Pennsylvania%27s_14th_congressional_district
Will get results for house/Missouri/District 2.html
Source: https://en.wikipedia.org/wiki/Missouri%27s_2nd_congressional_district
Will get results for house/California/District 11.html
Source: https://en.wikipedia.org/wiki/California%27s_11th_congressional_district
Will get results for house/Oklahoma/District 5.html
Source: https://en.wikipedia.org/wiki/Oklahoma%27s_5th_congressional_district
Will get results for house/California/District 4.html
Source: https://en.wikipedia.org/wiki/California%27s_4th_congressional_district
Will get results for house/North Carolina/District 5.html
Source

Will get results for house/Texas/District 35.html
Source: https://en.wikipedia.org/wiki/Texas%27s_35th_congressional_district
Will get results for house/Washington/District 8.html
Source: https://en.wikipedia.org/wiki/Washington%27s_8th_congressional_district
Will get results for house/Connecticut/District 1.html
Source: https://en.wikipedia.org/wiki/Connecticut%27s_1st_congressional_district
Will get results for house/Virginia/District 6.html
Source: https://en.wikipedia.org/wiki/Virginia%27s_6th_congressional_district
Will get results for house/Alabama/District 6.html
Source: https://en.wikipedia.org/wiki/Alabama%27s_6th_congressional_district
Will get results for house/Minnesota/District 7.html
Source: https://en.wikipedia.org/wiki/Minnesota%27s_7th_congressional_district
Will get results for house/California/District 15.html
Source: https://en.wikipedia.org/wiki/California%27s_15th_congressional_district
Will get results for house/Iowa/District 1.html
Source: https://en.wikipedia.o

Will get results for house/Colorado/District 6.html
Source: https://en.wikipedia.org/wiki/Colorado%27s_6th_congressional_district
Will get results for house/Kentucky/District 6.html
Source: https://en.wikipedia.org/wiki/Kentucky%27s_6th_congressional_district
Will get results for house/Virginia/District 9.html
Source: https://en.wikipedia.org/wiki/Virginia%27s_9th_congressional_district
Will get results for house/California/District 34.html
Source: https://en.wikipedia.org/wiki/California%27s_34th_congressional_district
Will get results for house/Alabama/District 2.html
Source: https://en.wikipedia.org/wiki/Alabama%27s_2nd_congressional_district
Will get results for house/New York/District 11.html
Source: https://en.wikipedia.org/wiki/New_York%27s_11th_congressional_district
Will get results for house/North Carolina/District 10.html
Source: https://en.wikipedia.org/wiki/North_Carolina%27s_10th_congressional_district
Will get results for house/Arizona/District 7.html
Source: https://en.

Will get results for house/California/District 17.html
Source: https://en.wikipedia.org/wiki/California%27s_17th_congressional_district
Will get results for house/New Mexico/District 3.html
Source: https://en.wikipedia.org/wiki/New_Mexico%27s_3rd_congressional_district
Will get results for house/California/District 33.html
Source: https://en.wikipedia.org/wiki/California%27s_33rd_congressional_district
Will get results for house/Texas/District 14.html
Source: https://en.wikipedia.org/wiki/Texas%27s_14th_congressional_district
Will get results for house/Illinois/District 1.html
Source: https://en.wikipedia.org/wiki/Illinois%27s_1st_congressional_district
Will get results for house/North Carolina/District 1.html
Source: https://en.wikipedia.org/wiki/North_Carolina%27s_1st_congressional_district
Will get results for house/California/District 24.html
Source: https://en.wikipedia.org/wiki/California%27s_24th_congressional_district
Will get results for house/New York/District 6.html
Source: 

Will get results for house/Texas/District 6.html
Source: https://en.wikipedia.org/wiki/Texas%27s_6th_congressional_district
Will get results for house/New York/District 16.html
Source: https://en.wikipedia.org/wiki/New_York%27s_16th_congressional_district
Will get results for house/California/District 45.html
Source: https://en.wikipedia.org/wiki/California%27s_45th_congressional_district
Will get results for house/Oklahoma/District 2.html
Source: https://en.wikipedia.org/wiki/Oklahoma%27s_2nd_congressional_district
Will get results for house/Pennsylvania/District 15.html
Source: https://en.wikipedia.org/wiki/Pennsylvania%27s_15th_congressional_district
Will get results for house/New Jersey/District 12.html
Source: https://en.wikipedia.org/wiki/New_Jersey%27s_12th_congressional_district
Will get results for house/Oregon/District 1.html
Source: https://en.wikipedia.org/wiki/Oregon%27s_1st_congressional_district
Will get results for house/Pennsylvania/District 11.html
Source: https://en.

**We now have on disk ALL the available historical district results from Wikipedia**

Lets take a look:

In [10]:
test_df = pd.read_csv('data/wikipedia/house_results.csv', index_col=0)
display(test_df.head())

,district,is_incumbent,name,party,percent,state,votes,won,year
0,District 12,NaN,Kay Granger,R,69.40,Texas,196482.0,1,2016
1,District 12,NaN,Bill Bradshaw,D,26.85,Texas,76029.0,0,2016
2,District 12,1.0,Kay Granger,R,71.31,Texas,113186.0,1,2014
3,District 12,0.0,Mark Greene,D,26.31,Texas,41757.0,0,2014
4,District 12,NaN,Kay Granger,R,70.90,Texas,175649.0,1,2012


## ballotpedia.org

So far so good but the 2018 results are missing on Wikipedia and the available data are not always exhaustives. So I decided to get the same informations from a different source: Ballotpedia.  
Here we have the complete 2018 results as well as historical date from 2012.  
Note that the incumbent information is consistent.  

### Get the House and Senate election result pages for all the available years

In [11]:
def get_house_senate_state_list():
    house_state_list = []
    senate_state_list = []
    filename = 'data/ballotpedia/house_state_list_src.html'
    url = 'https://ballotpedia.org/U.S._House_battlegrounds,_2018'
    
    # Check if the page has been stored on disk
    soup = check_or_save_page(filename, url)
    
    # Find the list of the U.S. Senate Elections by State (2018) pages
    table = soup.find('table', { 'class': 'infobox' })
    for link in table.find_all('a', href=lambda x: x and '/United_States_Senate_election_in_' in x):
        senate_state_list.append({
            'state': link.text,
            'url': 'https://ballotpedia.org{}'.format(link['href'])
        })

    # Find the list of the U.S. House Elections by State (2018) pages
    table = soup.find('table', { 'class': 'infobox' })
    for link in table.find_all('a', href=lambda x: x and (
        '/United_States_House_of_Representatives_election_in_' in x or
        '/United_States_House_of_Representatives_elections_in_' in x
    )):
        house_state_list.append({
            'state': link.text,
            'url': 'https://ballotpedia.org{}'.format(link['href'])
        })

    return house_state_list, senate_state_list

def get_district_pages(dict_page_url, year, state, district):
    """ 
        Recursively get all available previous election result pages
        for a given district
    """
    print('Will get house/{}/{}/{}.html'.format(state, district, year))
    # Create the directories if necessary
    if not os.path.exists('data/ballotpedia/house/'):
        os.makedirs('data/ballotpedia/house/')
    if not os.path.exists('data/ballotpedia/house/{}'.format(state)):
        os.makedirs('data/ballotpedia/house/{}'.format(state))
    if not os.path.exists('data/ballotpedia/house/{}/{}'.format(state, district)):
        os.makedirs('data/ballotpedia/house/{}/{}'.format(state, district))
    
    filename = 'data/ballotpedia/house/{}/{}/{}.html'.format(state, district, year)
    dict_soup = check_or_save_page(filename, dict_page_url)
    
    # Check if there is a link to a previous electoral year for this state
    table = dict_soup.find('table', { 'class': 'infobox' })
    div = table.find('div', style=lambda x: x and '#A3B1BF' in x and 'float:left;' in x)
            
    # If there is one
    if div is not None:
        # Extract the link election year
        prev_year = int(re.match(r'.*([1-2][0-9]{3})', div.text).group(1))
        
        if prev_year < year:                
            # Get the link to this disctict House election results parge
            link = div.find('a')
            #print(link['href'])

            # Get this page
            url = 'https://ballotpedia.org{}'.format(link['href'])
            get_district_pages(url, prev_year, state, district)

def get_house_senate_state_districts_list(house_state_list):
    start_year = 2018
    state_district_list = []
    for house_state in house_state_list:
        # To remove
        #if house_state['state'] != 'Maryland':
        #    continue

        filename = 'data/ballotpedia/2018_house_{}.html'.format(house_state['state'])
        
        # Check if the page has been stored on disk
        soup = check_or_save_page(filename, house_state['url'])
        #print(soup)
        
        # Get the district page links
        table = soup.find('table', { 'class': 'infobox' })
        
        links = table.find_all('a', href=lambda x: x and (
            '_Congressional_District_election,_' in x
        ))
        
        if len(links) == 0:
            title = soup.find('b', text=lambda x : x and 'District Pages' in x)
            links = title.parent.parent.find_all('a', href=lambda x: x and (
                '_Congressional_District_election,_' in x
            ))

        for link in links:
            print(link.text)
            url = 'https://ballotpedia.org{}'.format(link['href'])
            state_district_list.append({
                'state': house_state['state'],
                'district': link.text
            })
            #print(' |-', url)

            # Get the page
            get_district_pages(url, start_year, house_state['state'], link.text)
            
    return state_district_list

house_state_list, senate_state_list = get_house_senate_state_list()
state_district_list = get_house_senate_state_districts_list(house_state_list)

District 1
Will get house/Alabama/District 1/2018.html
Will get house/Alabama/District 1/2016.html
Will get house/Alabama/District 1/2014.html
Will get house/Alabama/District 1/2012.html
District 2
Will get house/Alabama/District 2/2018.html
Will get house/Alabama/District 2/2016.html
Will get house/Alabama/District 2/2014.html
Will get house/Alabama/District 2/2012.html
District 3
Will get house/Alabama/District 3/2018.html
Will get house/Alabama/District 3/2016.html
Will get house/Alabama/District 3/2014.html
Will get house/Alabama/District 3/2012.html
District 4
Will get house/Alabama/District 4/2018.html
Will get house/Alabama/District 4/2016.html
Will get house/Alabama/District 4/2014.html
Will get house/Alabama/District 4/2012.html
District 5
Will get house/Alabama/District 5/2018.html
Will get house/Alabama/District 5/2016.html
Will get house/Alabama/District 5/2014.html
Will get house/Alabama/District 5/2012.html
District 6
Will get house/Alabama/District 6/2018.html
Will get h

Will get house/California/District 22/2012.html
District 23
Will get house/California/District 23/2018.html
Will get house/California/District 23/2016.html
Will get house/California/District 23/2014.html
Will get house/California/District 23/2012.html
District 24
Will get house/California/District 24/2018.html
Will get house/California/District 24/2016.html
Will get house/California/District 24/2014.html
Will get house/California/District 24/2012.html
District 25
Will get house/California/District 25/2018.html
Will get house/California/District 25/2016.html
Will get house/California/District 25/2014.html
Will get house/California/District 25/2012.html
District 26
Will get house/California/District 26/2018.html
Will get house/California/District 26/2016.html
Will get house/California/District 26/2014.html
Will get house/California/District 26/2012.html
District 27
Will get house/California/District 27/2018.html
Will get house/California/District 27/2016.html
Will get house/California/Di

Will get house/Connecticut/District 3/2016.html
Will get house/Connecticut/District 3/2014.html
Will get house/Connecticut/District 3/2012.html
District 4
Will get house/Connecticut/District 4/2018.html
Will get house/Connecticut/District 4/2016.html
Will get house/Connecticut/District 4/2014.html
Will get house/Connecticut/District 4/2012.html
District 5
Will get house/Connecticut/District 5/2018.html
Will get house/Connecticut/District 5/2016.html
Will get house/Connecticut/District 5/2014.html
Will get house/Connecticut/District 5/2012.html
General election
Will get house/Delaware/General election/2018.html
Will get house/Delaware/General election/2016.html
Will get house/Delaware/General election/2014.html
Will get house/Delaware/General election/2012.html
District 1
Will get house/Florida/District 1/2018.html
Will get house/Florida/District 1/2016.html
Will get house/Florida/District 1/2014.html
Will get house/Florida/District 1/2012.html
District 2
Will get house/Florida/District

District 13
Will get house/Georgia/District 13/2018.html
Will get house/Georgia/District 13/2016.html
Will get house/Georgia/District 13/2014.html
Will get house/Georgia/District 13/2012.html
District 14
Will get house/Georgia/District 14/2018.html
Will get house/Georgia/District 14/2016.html
Will get house/Georgia/District 14/2014.html
Will get house/Georgia/District 14/2012.html
District 1
Will get house/Hawaii/District 1/2018.html
Will get house/Hawaii/District 1/2016.html
Will get house/Hawaii/District 1/2014.html
Will get house/Hawaii/District 1/2012.html
District 2
Will get house/Hawaii/District 2/2018.html
Will get house/Hawaii/District 2/2016.html
Will get house/Hawaii/District 2/2014.html
Will get house/Hawaii/District 2/2012.html
District 1
Will get house/Idaho/District 1/2018.html
Will get house/Idaho/District 1/2016.html
Will get house/Idaho/District 1/2014.html
Will get house/Idaho/District 1/2012.html
District 2
Will get house/Idaho/District 2/2018.html
Will get house/Ida

Will get house/Kentucky/District 3/2012.html
District 4
Will get house/Kentucky/District 4/2018.html
Will get house/Kentucky/District 4/2016.html
Will get house/Kentucky/District 4/2014.html
Will get house/Kentucky/District 4/2012.html
District 5
Will get house/Kentucky/District 5/2018.html
Will get house/Kentucky/District 5/2016.html
Will get house/Kentucky/District 5/2014.html
Will get house/Kentucky/District 5/2012.html
District 6
Will get house/Kentucky/District 6/2018.html
Will get house/Kentucky/District 6/2016.html
Will get house/Kentucky/District 6/2014.html
Will get house/Kentucky/District 6/2012.html
District 1
Will get house/Louisiana/District 1/2018.html
Will get house/Louisiana/District 1/2016.html
Will get house/Louisiana/District 1/2014.html
Will get house/Louisiana/District 1/2012.html
District 2
Will get house/Louisiana/District 2/2018.html
Will get house/Louisiana/District 2/2016.html
Will get house/Louisiana/District 2/2014.html
Will get house/Louisiana/District 2/20

...

District 1
Will get house/New Mexico/District 1/2018.html
Will get house/New Mexico/District 1/2016.html
Will get house/New Mexico/District 1/2014.html
Will get house/New Mexico/District 1/2012.html
District 2
Will get house/New Mexico/District 2/2018.html
Will get house/New Mexico/District 2/2016.html
Will get house/New Mexico/District 2/2014.html
Will get house/New Mexico/District 2/2012.html
District 3
Will get house/New Mexico/District 3/2018.html
Will get house/New Mexico/District 3/2016.html
Will get house/New Mexico/District 3/2014.html
Will get house/New Mexico/District 3/2012.html
District 1
Will get house/New York/District 1/2018.html
Will get house/New York/District 1/2016.html
Will get house/New York/District 1/2014.html
Will get house/New York/District 1/2012.html
District 2
Will get house/New York/District 2/2018.html
Will get house/New York/District 2/2016.html
Will get house/New York/District 2/2014.html
Will get house/New York/District 2/2012.html
District 3
Will get h

Will get house/North Carolina/District 12/2016.html
Will get house/North Carolina/District 12/2014.html
Will get house/North Carolina/District 12/2012.html
District 13
Will get house/North Carolina/District 13/2018.html
Will get house/North Carolina/District 13/2016.html
Will get house/North Carolina/District 13/2014.html
Will get house/North Carolina/District 13/2012.html
General election
Will get house/North Dakota/General election/2018.html
Will get house/North Dakota/General election/2016.html
Will get house/North Dakota/General election/2014.html
Will get house/North Dakota/General election/2012.html
District 1
Will get house/Ohio/District 1/2018.html
Will get house/Ohio/District 1/2016.html
Will get house/Ohio/District 1/2014.html
Will get house/Ohio/District 1/2012.html
District 2
Will get house/Ohio/District 2/2018.html
Will get house/Ohio/District 2/2016.html
Will get house/Ohio/District 2/2014.html
Will get house/Ohio/District 2/2012.html
District 3
Will get house/Ohio/Distri

Will get house/Pennsylvania/District 15/2014.html
Will get house/Pennsylvania/District 15/2012.html
District 16
Will get house/Pennsylvania/District 16/2018.html
Will get house/Pennsylvania/District 16/2016.html
Will get house/Pennsylvania/District 16/2014.html
Will get house/Pennsylvania/District 16/2012.html
District 17
Will get house/Pennsylvania/District 17/2018.html
Will get house/Pennsylvania/District 17/2016.html
Will get house/Pennsylvania/District 17/2014.html
Will get house/Pennsylvania/District 17/2012.html
District 18
Will get house/Pennsylvania/District 18/2018.html
District 1
Will get house/Rhode Island/District 1/2018.html
Will get house/Rhode Island/District 1/2016.html
Will get house/Rhode Island/District 1/2014.html
Will get house/Rhode Island/District 1/2012.html
District 2
Will get house/Rhode Island/District 2/2018.html
Will get house/Rhode Island/District 2/2016.html
Will get house/Rhode Island/District 2/2014.html
Will get house/Rhode Island/District 2/2012.html


Will get house/Texas/District 21/2016.html
Will get house/Texas/District 21/2014.html
Will get house/Texas/District 21/2012.html
District 22
Will get house/Texas/District 22/2018.html
Will get house/Texas/District 22/2016.html
Will get house/Texas/District 22/2014.html
Will get house/Texas/District 22/2012.html
District 23
Will get house/Texas/District 23/2018.html
Will get house/Texas/District 23/2016.html
Will get house/Texas/District 23/2014.html
Will get house/Texas/District 23/2012.html
District 24
Will get house/Texas/District 24/2018.html
Will get house/Texas/District 24/2016.html
Will get house/Texas/District 24/2014.html
Will get house/Texas/District 24/2012.html
District 25
Will get house/Texas/District 25/2018.html
Will get house/Texas/District 25/2016.html
Will get house/Texas/District 25/2014.html
Will get house/Texas/District 25/2012.html
District 26
Will get house/Texas/District 26/2018.html
Will get house/Texas/District 26/2016.html
Will get house/Texas/District 26/2014

Will get house/West Virginia/District 2/2012.html
District 3
Will get house/West Virginia/District 3/2018.html
Will get house/West Virginia/District 3/2016.html
Will get house/West Virginia/District 3/2014.html
Will get house/West Virginia/District 3/2012.html
District 1
Will get house/Wisconsin/District 1/2018.html
Will get house/Wisconsin/District 1/2016.html
Will get house/Wisconsin/District 1/2014.html
Will get house/Wisconsin/District 1/2012.html
District 2
Will get house/Wisconsin/District 2/2018.html
Will get house/Wisconsin/District 2/2016.html
Will get house/Wisconsin/District 2/2014.html
Will get house/Wisconsin/District 2/2012.html
District 3
Will get house/Wisconsin/District 3/2018.html
Will get house/Wisconsin/District 3/2016.html
Will get house/Wisconsin/District 3/2014.html
Will get house/Wisconsin/District 3/2012.html
District 4
Will get house/Wisconsin/District 4/2018.html
Will get house/Wisconsin/District 4/2016.html
Will get house/Wisconsin/District 4/2014.html
Will 

### Extract the House election results for every districts and years

In [12]:
def extract_district_data(state_district_list):
    results = []
    undesirable_chars = ['\*', '%', 'Incumbent', '\(D\)', '\(R\)']
    for item in state_district_list:
        # To remove
        #if item['state'] != 'New Hampshire' or item['district'] != 'District 2':
        #if item['state'] != 'Wyoming':
        #    continue

        # Get the pages
        directory = 'data/ballotpedia/house/{}/{}'.format(item['state'], item['district'])
        files = [f for f in listdir(directory) if isfile(join(directory, f))]
        
        # For each year, get the district data
        for file in files:
            # Extract the year
            year = int(re.match(r'.*([1-2][0-9]{3})', file).group(1))
            candidate_rows = []
            
            # To remove
            #if year != 2018:
            #    continue
            
            # Get the page content
            filename = 'data/ballotpedia/house/{}/{}/{}'.format(item['state'], item['district'], file)
            #with open(filename) as my_file:
            with open(filename, encoding='utf-8') as my_file:
                soup = BeautifulSoup(my_file.read(), "html.parser")
            
            # The 2018 pages requires a different approach
            if year == 2018:
                #print(2018)
                # Find the result table
                table = soup.find('table',  { 'class': 'results_table' })
                rows = table.find_all('tr')
                
                for row in rows:
                    cols = row.find_all('td')
                    cols = [ele.text.strip() for ele in cols]
                    cols = [ele for ele in cols if ele] # Get rid of empty values
                    
                    # Check if is incumbant
                    incumbent = 1 if row.find('b') and row.find('b').find('u') else 0
                    #print(cols)
                    if len(cols) == 4 and cols[0] == '✔':
                        is_winner = 1
                        name = cols[1] +' Incumbent' if incumbent == 1 else cols[1]
                        percent = cols[2] if len(cols) > 1 else np.NaN
                        votes = cols[3] if len(cols) > 2 else np.NaN
                        party = 'Democratic' if '(D)' in cols[1] else 'Republican'
                        candidate_rows.append([party, name, percent, votes, is_winner])
                        
                    elif len(cols) == 3 and '(D)' in cols[0] or '(R)' in cols[0]:
                        is_winner = 0
                        name = cols[0] +' Incumbent' if incumbent == 1 else cols[0]
                        percent = cols[1] if len(cols) > 1 else np.NaN
                        
                        votes = cols[2] if len(cols) > 2 else np.NaN
                        if len(cols) > 1:
                            party = 'Democratic' if '(D)' in cols[0] else 'Republican'
                        else:
                            party = np.NaN
                            
                        candidate_rows.append([party, name, percent, votes, is_winner])
                
            else:            
                # Find the result table
                th = soup.find('th', colspan='5', style=lambda x: x and 'background-color:#444' in x)
                table = th.find_parent('table')
                #table_body = table.find('tbody')
                rows = table.find_all('tr')
                #print(rows)

                for row in rows:
                    cols = row.find_all('td')
                    cols = [ele.text.strip() for ele in cols]
                    cols = [ele for ele in cols if ele] # Get rid of empty values

                    # Ignore the rows not about the candidates
                    if 'Republican' not in cols and not 'Democratic' in cols:
                        continue

                    # Check if the candidate won the elections
                    is_winner = 1 if row.find('a', title="Won") else 0
                    cols.append(is_winner)
                    candidate_rows.append(cols)
                    
            # If there was only one candidate
            if len(candidate_rows) == 1:
                if type(candidate_rows[0][3]) is int:
                    candidate_rows[0].append(candidate_rows[0][3])
                    candidate_rows[0][3] = np.NaN

            for candidate in candidate_rows:
                #print(year, item['district'], candidate)
                
                # Get and format the candidate party
                candidate_party = 'R' if candidate[0] == 'Republican' else 'D'
                
                # Get and clean the candidate name
                candidate_name = re.sub("|".join(undesirable_chars), "", candidate[1]).rstrip()
                
                # Get and clean the candidate percent
                if type(candidate[2]) is str:
                    candidate_percent = float(candidate[2].replace('%', ''))
                else:
                    candidate_percent = candidate[2]
                
                # Get and clean the candidate vote
                if type(candidate[3]) is str:
                    candidate_vote = int(candidate[3].replace(',', ''))
                else:
                    candidate_vote = candidate[3]
                
                # Determine whether or not the candidate is incumbent
                candidate_is_incumbent = 1 if 'Incumbent' in candidate[1] else 0
                
                results.append({
                    'year': year,
                    'state': item['state'],
                    'district': item['district'] if item['district'] != 'General election' else 'At-Large',
                    'name': candidate_name,
                    'party': candidate_party,
                    'percent': candidate_percent,
                    'votes': candidate_vote,
                    'is_incumbent': candidate_is_incumbent,
                    'won': candidate[4]
                })
                
                #print(results)
                #print('')

        #soup = BeautifulSoup(my_file.read(), "html.parser")
        #print(soup)
    return results

ballo_house_history = extract_district_data(state_district_list)

# Store on disk
ballo_house_history_df = pd.DataFrame(ballo_house_history)
ballo_house_history_df.to_csv('data/ballotpedia/ballo_results.csv', encoding='utf-8')

## Merge the data from wikipedia.org and  ballotpedia.org

Now we have two dataset with the same columns and some overlaping data. Its time to merge them.  
It appears that the data from ballotpedia.org are more consistent so we will favor them.

In [13]:
ballo_df = pd.read_csv('data/ballotpedia/ballo_results.csv', index_col=0)
wikipedia_df = pd.read_csv('data/wikipedia/house_results.csv', index_col=0)

merged_df =  pd.concat([wikipedia_df.loc[wikipedia_df['year'] < 2012], ballo_df])
display(merged_df.head())

,district,is_incumbent,name,party,percent,state,votes,won,year
6,District 12,NaN,Kay Granger,R,71.85,Texas,109882.0,1,2010
7,District 12,NaN,Tracey Smith,D,25.13,Texas,38434.0,0,2010
8,District 12,NaN,Kay Granger,R,67.59,Texas,181662.0,1,2008
9,District 12,NaN,Tracey Smith,D,30.60,Texas,82250.0,0,2008
10,District 12,NaN,Kay Granger,R,66.94,Texas,98371.0,1,2006


## Imput and derive from the data

From this data, we can create new predictors: 

- Impute the missing data for `is_incumbent`
- First time the incumbent has been elected
- Number of incumbents victories

In [14]:
derived_df = merged_df.copy()
derived_df = derived_df.sort_values(by=['year'])

# Number of NaN values for the `is_incumbent` col
print('NaN is_incumbent values before:', derived_df['is_incumbent'].isna().sum())

def check_if_is_incumbent(row):
    is_incumbent = row['is_incumbent']

    # Check if there is a previous election for this state, district and candidate
    prev_year = row['year'] - 2
    prev_year_row = derived_df.loc[(derived_df['state'] == row['state']) & (derived_df['district'] == row['district']) & (derived_df['name'] == row['name']) & (derived_df['year'] == prev_year)]

    # If the row has NaN for the col `is_incumbent` and the candidate won the last election
    if np.isnan(row['is_incumbent']) and prev_year_row.empty is False and prev_year_row['won'].values[0] == 1:
        is_incumbent = 1
    # If the candidate lose the last elections, it is likely he isn't the incumbent
    elif np.isnan(row['is_incumbent']) and prev_year_row.empty is False and prev_year_row['won'].values[0] == 0:
        is_incumbent
    # If the candidate didn't participate to the last election, 
    # we can safely assume he isn't the incumbent
    elif np.isnan(row['is_incumbent']) and prev_year_row.empty:
        is_incumbent = 0
        
    return is_incumbent

# Check if the candidate has already been elected the past year (and so is an incumbent)
derived_df['is_incumbent'] = derived_df.apply(check_if_is_incumbent, axis=1)

print('NaN is_incumbent values after:', derived_df['is_incumbent'].isna().sum())

NaN is_incumbent values before: 2748
NaN is_incumbent values after: 118


We gain the `is_incumbent` information for 2630 rows. 118 remains NaN.  
Now, we will add a new column to know the year of the first election the candidate won:

In [15]:
def get_first_year_elected(row):
    first_elected = np.NaN
    # Get the first year the candidate has been elected (if exist)
    victories = derived_df.loc[(derived_df['state'] == row['state']) & (derived_df['district'] == row['district']) & (derived_df['name'] == row['name']) & (derived_df['won'] == 1)]

    if victories.empty is False:
        first_elected = victories['year'].min()
        
    return first_elected

derived_df['first_time_elected'] = derived_df.apply(get_first_year_elected, axis=1)

Finally, we want to count the number of victories of each candidate:

In [16]:
def count_victories(row):
    count_victories = 0
    victories = derived_df.loc[(derived_df['state'] == row['state']) & (derived_df['district'] == row['district']) & (derived_df['name'] == row['name']) & (derived_df['won'] == 1)]
    
    if victories.empty is False:
        count_victories = len(victories)
    
    return count_victories
    
derived_df['count_victories'] = derived_df.apply(count_victories, axis=1)

In [17]:
display(derived_df.head())

,district,is_incumbent,name,party,percent,state,votes,won,year,first_time_elected,count_victories
1272,District 1,0.0,Ratliff Boon,D,42.1,Indiana,4281.0,1,1824,1824.0,7
1273,District 1,1.0,Ratliff Boon,D,42.8,Indiana,5202.0,1,1826,1824.0,7
1274,District 1,1.0,Ratliff Boon,D,52.2,Indiana,7272.0,1,1828,1824.0,7
1275,District 1,1.0,Ratliff Boon,D,50.9,Indiana,11280.0,1,1830,1824.0,7
1276,District 1,0.0,John Law,D,49.1,Indiana,10868.0,0,1830,1860.0,2


## Additional factors

Let's also add the **unemployement rate** at the district level when available, else, at the national level.

In [18]:
def get_district_number(row):
    district = row['district']
    
    p = re.compile("District (.*) \(")
    dist = p.search(row['district']).group(1)
    
    if dist == '(at Large)':
        district = 'At-Large'
    else:
        district = 'District {}'.format(dist)
    
    return district

def get_state_dist_unemployement(row):
    unemployement_rate = np.NaN
    
    # Find the corresponding unemployement rate
    unemp_row = unemp_df.loc[(unemp_df['state'] == row['state']) & \
                             (unemp_df['district'] == row['district']) & \
                             (unemp_df['year'] == row['year'])]

    if unemp_row.empty is False:
        unemployement_rate = unemp_row['unemp_rate_16'].values[0]
    else:
        # Use the national unemployement rate of October instead
        nat_oct_unemployement_rate = national_unemployement_rate.loc[national_unemployement_rate['Year'] == row['year']]['Oct']
        unemployement_rate = np.NaN if nat_oct_unemployement_rate.empty else nat_oct_unemployement_rate.values[0]
        
    return unemployement_rate

augmented_df = derived_df.copy()

unemp_df = pd.read_csv('data/unemployment/unemp_2012_2017.csv',sep=';')
unemp_df['state'] = unemp_df['state'].str.rsplit(',').str[-1].str.strip()
unemp_df['district'] = unemp_df.apply(get_district_number, axis=1)

augmented_df['unemployement_rate'] = augmented_df.apply(get_state_dist_unemployement, axis=1)

display(augmented_df.loc[augmented_df['state'] == 'Alabama'].head())

,district,is_incumbent,name,party,percent,state,votes,won,year,first_time_elected,count_victories,unemployement_rate
5040,District 3,0.0,Joe Turnham,D,48.00,Alabama,87351.0,0,2002,NaN,0,5.7
5039,District 3,0.0,Mike D. Rogers,R,50.00,Alabama,91169.0,1,2002,2002.0,5,5.7
3552,District 6,0.0,Spencer T. Bachus III,R,90.00,Alabama,178171.0,1,2002,2002.0,5,5.7
4859,District 2,0.0,Charles Wood,D,29.18,Alabama,64958.0,0,2002,NaN,0,5.7
4858,District 2,0.0,Terry Everett,R,68.20,Alabama,151830.0,1,2002,2002.0,3,5.7


Fundraising

In [19]:
df_fund = pd.read_csv('data/Fundraising/followthemoney_2009-2018.csv')

df_fund['state_abbr']=df_fund['Office_Sought'].str.split(' ', expand=True)[3]
df_fund['district']=df_fund['Office_Sought'].str.split(' ', expand=True)[4].astype(int)
df_fund['district']='District '+df_fund['district'].astype(str)
df_fund['General_Party']=df_fund['General_Party'].str.slice(0,1) #['D', 'R', 'T', 'U']

states = pd.read_csv('data/states.csv')

df_fund=df_fund.join(states.set_index('Abbreviation'), on='state_abbr')
df_fund_grouped=df_fund.groupby(['Election_Year:id', 'State', 'district', 'General_Party'])['Total_$'].sum().reset_index(drop=False)

idx = df_fund_grouped.groupby(['Election_Year:id', 'State', 'district'])['Total_$'].transform(max) == df_fund_grouped['Total_$']

df_fund_rich_party=df_fund_grouped[idx]
df_fund_rich_party.columns=['year', 'state', 'district', 'rich party', 'Total_$']
df_fund_rich_party=df_fund_rich_party.drop('Total_$', axis=1)

augmented_df=augmented_df.join(df_fund_rich_party.set_index(['year', 'state', 'district']), on=['year', 'state', 'district']).copy()
augmented_df.head()

,district,is_incumbent,name,party,percent,state,votes,won,year,first_time_elected,count_victories,unemployement_rate,rich party
1272,District 1,0.0,Ratliff Boon,D,42.1,Indiana,4281.0,1,1824,1824.0,7,NaN,NaN
1273,District 1,1.0,Ratliff Boon,D,42.8,Indiana,5202.0,1,1826,1824.0,7,NaN,NaN
1274,District 1,1.0,Ratliff Boon,D,52.2,Indiana,7272.0,1,1828,1824.0,7,NaN,NaN
1275,District 1,1.0,Ratliff Boon,D,50.9,Indiana,11280.0,1,1830,1824.0,7,NaN,NaN
1276,District 1,0.0,John Law,D,49.1,Indiana,10868.0,0,1830,1860.0,2,NaN,NaN


We will also add the following national level factors:  

- is_presidential_year: 1 if Yes, 0 if No
- president_can_be_re_elected: Can the president stand for re-election ? 1 = Yes, 0 = No  
- president_party: D or R
- president_overall_avg_job_approval: Only available from 1953 to 2018
- last_D_house_seats: # of Democrat seats at the last elections
- last_R_house_seats: # of Republican seats at the last elections
- last_house_majority: Which party had the majority at the last House elections. D or R

In [20]:
nat_augmented_df = augmented_df.copy()

def add_is_presidential_year(row):
    result = np.NaN
    df = national_level_factors.loc[national_level_factors['year'] == row['year'], 'is_presidential_year']
    
    if df.empty is False:
        result = df.values[0]
    
    return result

def add_president_can_be_re_elected(row):
    # idx = (np.abs(president_elected_history['year'].values-row['year']+1)).argmin()
    # return president_elected_history['can_be_re_elected'].loc[[idx]].values[0]
    result = np.NaN
    df = national_level_factors.loc[national_level_factors['year'] == row['year'], 'president_can_be_re_elected']
    
    if df.empty is False:
        result = df.values[0]
    
    return result

def add_president_party(row):
    # idx = (np.abs(president_elected_history['year'].values-row['year']+1)).argmin()
    # return president_elected_history['president_elected_party'].loc[[idx]].values[0]
    result = np.NaN
    df = national_level_factors.loc[national_level_factors['year'] == row['year'], 'president_party']
    
    if df.empty is False:
        result = df.values[0]
    
    return result

def add_president_overall_avg_job_approval(row):
    # idx = (np.abs(president_elected_history['year'].values-row['year']+1)).argmin()
    # president_name = president_elected_history['president_elected'].loc[[idx]].values[0]
    # president_overall_avg_job_approval = presidential_approval_df.loc[presidential_approval_df['PresidentName'] == president_name]['OverallAverage']
    # return float(president_overall_avg_job_approval.values[0])/100 if president_overall_avg_job_approval.values.size else np.NaN
    result = np.NaN
    df = national_level_factors.loc[national_level_factors['year'] == row['year'], 'president_overall_avg_job_approval']
    
    if df.empty is False:
        result = df.values[0]
    
    return result

def add_last_D_house_seats(row):
    result = np.NaN
    df = national_level_factors.loc[national_level_factors['year'] == row['year'], 'last_democrat_seats']
    
    if df.empty is False:
        result = df.values[0]
    
    return result
    
def add_last_R_house_seats(row):
    result = np.NaN
    df = national_level_factors.loc[national_level_factors['year'] == row['year'], 'last_republican seats']
    
    if df.empty is False:
        result = df.values[0]
    
    return result

def add_last_house_majority(row):
    result = np.NaN
    df = national_level_factors.loc[national_level_factors['year'] == row['year'], 'last_house_majority']
    
    if df.empty is False:
        result = df.values[0]
    
    return result

# is_presidential_year
nat_augmented_df['is_presidential_year'] = nat_augmented_df.apply(add_is_presidential_year, axis=1)

# president_can_be_re_elected
nat_augmented_df['president_can_be_re_elected'] = nat_augmented_df.apply(add_president_can_be_re_elected, axis=1)

# president_party
nat_augmented_df['president_party'] = nat_augmented_df.apply(add_president_party, axis=1)

# president_overall_avg_job_approval
nat_augmented_df['president_overall_avg_job_approval'] = nat_augmented_df.apply(add_president_overall_avg_job_approval, axis=1)

# last_D_house_seats
nat_augmented_df['last_D_house_seats'] = nat_augmented_df.apply(add_last_D_house_seats, axis=1)

# last_R_house_seats
nat_augmented_df['last_R_house_seats'] = nat_augmented_df.apply(add_last_R_house_seats, axis=1)

# last_house_majority
nat_augmented_df['last_house_majority'] = nat_augmented_df.apply(add_last_house_majority, axis=1)


In [21]:
display(nat_augmented_df.loc[nat_augmented_df['year'] == 1958].head())
#display(nat_augmented_df.head())

,district,is_incumbent,name,party,percent,state,votes,won,year,first_time_elected,count_victories,unemployement_rate,rich party,is_presidential_year,president_can_be_re_elected,president_party,president_overall_avg_job_approval,last_D_house_seats,last_R_house_seats,last_house_majority
4671,District 27,0.0,Robert M. Castle,R,27.7,California,40317.0,0,1958,NaN,0,6.7,NaN,0.0,0.0,R,0.65,232.0,203.0,D
5312,District 16,0.0,Melvin Lennard,D,42.2,California,51616.0,0,1958,NaN,0,6.7,NaN,0.0,0.0,R,0.65,232.0,203.0,D
5311,District 16,1.0,Donald L. Jackson,R,57.8,California,70724.0,1,1958,1948.0,6,6.7,NaN,0.0,0.0,R,0.65,232.0,203.0,D
743,District 3,0.0,Fred M. Betz,D,50.0,Colorado,63112.0,1,1958,1958.0,1,6.7,NaN,0.0,0.0,R,0.65,232.0,203.0,D
3726,District 12,0.0,Daniel K. Halpin,R,18.9,California,26228.0,0,1958,NaN,0,6.7,NaN,0.0,0.0,R,0.65,232.0,203.0,D


# Assemble the final dataset

In [22]:
national_level_factors

,year,is_presidential_year,president_party,president_can_be_re_elected,president_overall_avg_job_approval,oct_unemployement_rate,last_democrat_seats,last_republican seats,last_house_majority
0,1858,0,D,1,NaN,NaN,133,90,D
1,1860,1,R,1,NaN,NaN,98,116,R
2,1862,0,R,1,NaN,NaN,45,108,R
3,1864,1,R,0,NaN,NaN,72,87,R
4,1866,0,R,0,NaN,NaN,38,137,R
5,1868,1,R,1,NaN,NaN,47,175,R
6,1870,0,R,1,NaN,NaN,67,171,R
7,1872,1,R,0,NaN,NaN,104,139,R
8,1874,0,R,0,NaN,NaN,89,203,R
9,1876,1,R,1,NaN,NaN,183,106,D
